In [ ]:
##################################################################################
#
#       Make Heat Index
#       By Cascade Tuholske Spring 2021
#
#       Program makes daily maximum heat index tifs with CHIRTS-daily Tmax and 
#       relative humidity min estimated with CHIRTS-daily Tmax
#
#       NOAA Heat Index Equation - https://www.wpc.ncep.noaa.gov/html/heatindex_equation.shtml
#
#       Note - right now units are all in C. They can be updated to F or C as needed. 
#              See make_hi function to make changes CPT July 2021
#
#################################################################################

#### Dependencies
import numpy as np
import pandas as pd
import xarray 
import os
import glob
import rasterio
import matplotlib.pyplot as plt
import time
import multiprocessing as mp 
from multiprocessing import Pool
import multiprocessing
import ClimFuncs

In [ ]:
def parallel_loop(function, start_list, cpu_num):
    """Run a routine in parallel
    Args: 
        function = function to apply in parallel
        start_list = list of args to loop through in parallel
        cpu_num = numper of cpus to fire  
    """ 
    start = time.time()
    pool = Pool(processes = cpu_num)
    pool.map(function, start_list)
    # pool.map_async(function, dir_list)
    pool.close()

    end = time.time()
    print(end-start)

In [ ]:
def make_hi(zipped_fns):
    
    """ Takes an RH and Tmax file zipped together and writes a heat index tif
    Args:
        zipped_fns = zipped RH [0] and tmax [1] file path/name
        path_out = path/to/himax_out 
        data_out = string usuall himax
    """
    
    # print process
    print(multiprocessing.current_process(), year)
    p = multiprocessing.current_process()
    
    # get data date
    date =zipped_fns[0].split('RH.')[1].split('.tif')[0]
       
    # get meta data
    meta = rasterio.open(zipped_fns[0]).meta
    meta['dtype'] = 'float64'
    
    # make hi
    rh_fn = zipped_fns[0] 
    tmax_fn = zipped_fns[1]
    tmax = xarray.open_rasterio(tmax_fn)
    rh = xarray.open_rasterio(rh_fn)
    hi = ClimFuncs.heatindex(Tmax = tmax, RH = rh, unit_in = 'C', unit_out = 'C')
    
    # get array
    arr = hi[0]
    
    # Set path out
    himax_path = os.path.join('/scratch/cascade/UEH-daily/himax/', str(year)) #/csv # path to write out HImax daily tifs
    data_out = 'himax'
    fn_out = os.path.join(himax_path,data_out+'.'+date+'.tif')
    
    def write_raster(fn_out, **meta)
    
    with rasterio.open(fn_out, 'w', **meta) as out:
        out.write_band(1, arr)
    p.terminate()

In [ ]:
def hi_loop(year):
    
    """
    """
    
    # Set up file paths
    rh_path = os.path.join('/home/CHIRTS/daily_ERA5/w-ERA5_Td.eq2-2-spechum-Tmax/', str(year)) # RH min made with CHIRTS-daily Tmax
    tmax_path = os.path.join('/home/chc-data-out/products/CHIRTSdaily/v1.0/global_tifs_p05/Tmax/', str(year)) # CHIRTS-daily Tmax
    himax_path = os.path.join('/scratch/cascade/UEH-daily/himax/', str(year)) #/csv # path to write out HImax daily tifs
    
    # make dir to write files
    cmd = 'mkdir '+himax_path
    os.system(cmd)
    print(cmd, 'made')
    
    # zip RH and Tmax
    rh_fns = sorted(glob.glob(rh_path+'/*.tif'))
    tmax_fns = sorted(glob.glob(tmax_path+'/*.tif'))
    zipped_list = list(zip(rh_fns,tmax_fns))
    
    test = zipped_list[:4]
    
    # data out
    data = 'himax'
    
    # run in parallel
    pool = Pool(processes = 20)
    pool.map(make_hi, test)
    pool.close()

In [ ]:
# Make years 
year_list = list(range(1983,2016+1))
year_sub = year_list[0:4]

In [ ]:
# Run it
for year in year_sub:
    hi_loop(year)